In [1]:
import requests
import json
import stix2

def generate_dns_records(dns_records):
    stix_records = []
    for record in dns_records:
        if record['record_type'] == 'A':
            stix_record = stix2.IPv4Address(value=record['value'])
        elif record['record_type'] == 'AAAA':
            stix_record = stix2.IPv6Address(value=record['value'])
        elif record['record_type'] == 'MX':
            stix_record = stix2.EmailMessage(
                from_ref=stix2.EmailAddress(value='example.com'),
                to_refs=[stix2.EmailAddress(value='example.com')],
                subject='MX Record',
                body='Priority: ' + str(record['priority']) + '\nValue: ' + record['value']
            )
        elif record['record_type'] == 'NS':
            stix_record = stix2.DomainName(value=record['value'])
        elif record['record_type'] == 'SOA':
            stix_record = stix2.DomainName(
                value=record['mname'],
                custom_properties={
                    'rname': record['rname'],
                    'serial': record['serial'],
                    'refresh': record['refresh'],
                    'retry': record['retry'],
                    'expire': record['expire'],
                    'minimum': record['ttl']
                }
            )
        elif record['record_type'] == 'TXT':
            stix_record = stix2.Artifact(
                mime_type='text/plain',
                payload=record['value']
            )
        else:
            # unsupported record type
            continue
        stix_records.append(stix_record)
    return stix_records

# set the domain and API key
domain = 'ualr.edu'
api_key = 'wHwJqDW0sOyr+YT8s31SQQ==6BkjC2ZIEvIGDjyi'

# create the API URL with the domain and API key
api_url = 'https://api.api-ninjas.com/v1/dnslookup?domain={}'.format(domain)
headers = {'X-Api-Key': api_key}

# make the API request
response = requests.get(api_url, headers=headers)

if response.status_code == requests.codes.ok:
    # parse the JSON response into a dictionary
    result = json.loads(response.text)

    if 'records' in result:
        dns_records = result['records']
        # create a STIX 2.1 Domain Name object
        domain_obj = {
            "type": "domain-name",
            "value": domain,
            "resolves_to_refs": generate_dns_records(dns_records)
        }

        # print the STIX object as JSON
        print(json.dumps(domain_obj, indent=4))
    else:
        print(response.text)
else:
    print("Error:", response.status_code, response.text)


[{"record_type": "A", "value": "144.167.4.20"}, {"record_type": "MX", "priority": 10, "value": "ALT4.ASPMX.L.GOOGLE.COM."}, {"record_type": "MX", "priority": 5, "value": "ALT2.ASPMX.L.GOOGLE.COM."}, {"record_type": "MX", "priority": 5, "value": "ALT1.ASPMX.L.GOOGLE.COM."}, {"record_type": "MX", "priority": 10, "value": "ALT3.ASPMX.L.GOOGLE.COM."}, {"record_type": "MX", "priority": 1, "value": "ASPMX.L.GOOGLE.COM."}, {"record_type": "NS", "value": "ns7.net.ualr.edu."}, {"record_type": "NS", "value": "ns3.ualr.edu."}, {"record_type": "NS", "value": "ns2.ualr.edu."}, {"record_type": "NS", "value": "ns4.ualr.edu."}, {"record_type": "NS", "value": "ns.ualr.edu."}, {"record_type": "NS", "value": "ns5.ualr.edu."}, {"record_type": "NS", "value": "ns6.ualr.edu."}, {"record_type": "SOA", "mname": "ns.ualr.edu.", "rname": "postmaster.ualr.edu.", "serial": 2023032587, "refresh": 10800, "retry": 3600, "expire": 604800, "ttl": 38400}, {"record_type": "TXT", "value": "ZOOM_verify_FtfVtPKpSoKiui1hCYYJ